In [90]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, classification_report
import matplotlib.pylab as plt
import datetime as dt
import time

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.recurrent import LSTM, GRU
from keras.layers import Convolution1D, MaxPooling1D

%matplotlib inline  

In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd


# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    #df.fillna(0, inplace=True)
    df.dropna(inplace=True)
    return df

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	for i in range(nb_epoch):
		model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
		model.reset_states()
	return model

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

def parser(x):
    return pd.datetime.strptime(x, '%Y.%m.%d')

Using TensorFlow backend.


### Import file

In [4]:
df = pd.read_csv('EURUSD.csv', header=None, 
                  names = ['date', 'dummy', 'open', 'high', 'low','close','volume'], parse_dates=[0], date_parser=parser)

#drop columns
df.drop(df.loc[:,['dummy','high','low','volume']].head(0).columns, axis=1, inplace=True)

#create day of week field
df['weekday']= df['date'].dt.dayofweek
df.head()
print (df['weekday'].value_counts())

#select from Monday to Friday only
df1=df.loc[df['weekday'].isin([0,1,2,3,4])]

#distribution by field
print (df1['weekday'].value_counts())

df1['close_open']=df1['close'] - df1['open']

print (df1.head())

2    490
0    489
1    488
3    487
4    487
6    245
Name: weekday, dtype: int64
2    490
0    489
1    488
3    487
4    487
Name: weekday, dtype: int64
        date    open   close  weekday  close_open
0 2007-12-17  1.4390  1.4400        0      0.0010
1 2007-12-18  1.4399  1.4413        1      0.0014
2 2007-12-19  1.4414  1.4383        2     -0.0031
3 2007-12-20  1.4384  1.4330        3     -0.0054
4 2007-12-21  1.4329  1.4357        4      0.0028


/Users/kelvidpang/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Model

In [7]:
raw_values = df1['close'].values
supervised = timeseries_to_supervised(raw_values, 1)
supervised_values = supervised.values
train, test = supervised_values[0:501], supervised_values[501]
train_scaled=train
test_scaled=test

In [ ]:
#nb_epoch_range=range(100,1100,100)
nb_epoch_range=[1]
neuron_range=[1]
for iepoch in nb_epoch_range:
    for ineuron in neuron_range:
        # fit the model
        lstm_model = fit_lstm(train_scaled, 1, iepoch, ineuron)
        # walk-forward validation on the test data
        X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
        yhat = forecast_lstm(lstm_model, 1, X)
        expected = raw_values[len(train) + i]
        

print("done")